<a href="https://colab.research.google.com/github/damylusi/Hamoye-Internship/blob/main/Hamoye_Stage_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy as sp

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
from google.colab import drive
drive.mount("/content/Data")

Mounted at /content/Data


# MACHINE LEARNING: REGRESSION - PREDICTING ENERGY EFFICIENCY OF BUILDINGS 

In [ ]:
energy = pd.read_csv("/content/Data/MyDrive/energydata_complete.csv", encoding = "ISO-8859-1")

# Question 12

From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

x = energy[['T2']]
y = energy[['T6']]

linear_model = LinearRegression()
linear_model.fit(x,y)

pred = linear_model.predict(x)
round(r2_score(y, pred), 2)



0.64

In [ ]:
scaler = MinMaxScaler()

energy = energy.drop(columns=['date', 'lights'], axis = 1)

normalize_energy = pd.DataFrame(scaler.fit_transform(energy), columns=energy.columns)

feat = normalize_energy.drop(columns=['Appliances'])
target = normalize_energy['Appliances']

x_train, x_test, y_train, y_test = train_test_split(feat, target, test_size=0.3, random_state=42)

#create Regression
regr = LinearRegression()

#fit the linear regression
regr.fit(x_train, y_train)

#obtain predictions
predict= regr.predict(x_test)






# Question 13

Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [ ]:
MAE = mean_absolute_error(y_test, predict)
round(MAE, 2)

0.05

# Question 14

What is the Residual Sum of Squares (in two decimal places)?

In [ ]:
RSS = np.sum(np.square(y_test - predict))
round(RSS, 2)

45.35

# Question 15

What is the Root Mean Squared Error (in three decimal places)?



In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test, predict))
round(RMSE, 3)

0.088

# Question 16

What is the Coefficient of Determination (in two decimal places)?

In [ ]:
#R Squared is Coefficient of Determination

r_squared = r2_score(y_test, predict)
round(r_squared, 2)

0.15

# Question 17

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [ ]:
def get_weights_energy(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_energy = pd.DataFrame(weights).reset_index()
    weights_energy.columns = ['Feat', col_name]
    weights_energy[col_name].round(3)
    return weights_energy

linear_model_weights = get_weights_energy(regr, x_train, 'Linear_Model_Weight')
linear_model_weights

,Feat,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


# Question 18

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [ ]:
ridge = Ridge(alpha=0.4)
ridge.fit(x_train, y_train)
ridge_prediction = ridge.predict(x_test)
train_score = ridge.score(x_train, y_train)
test_score = ridge.score(x_test, y_test)

RMSE = np.sqrt(mean_squared_error(y_test, ridge_prediction))
print(round(RMSE, 3))

0.088


# Question 19

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [ ]:
#lasso regression

lasso_regression = Lasso(alpha = 0.001)
lasso_regression.fit(x_train, y_train)
lasso_prediction = lasso_regression.predict(x_test)


lasso_weights_energy= get_weights_energy(lasso_regression, x_train, 'Lasso_weight')
lasso_weights_energy


,Feat,Lasso_weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


# Question 20

What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)



In [ ]:
RMSE = np.sqrt(mean_squared_error(y_test, lasso_prediction))
round(RMSE, 3)

0.094